In [1]:
#udacity !pip -q install ./python

In [2]:
import sys, os
sys.path.append(os.getcwd())
print(sys.path)

['/Users/chrisg/miniconda3/envs/drlnd/lib/python36.zip', '/Users/chrisg/miniconda3/envs/drlnd/lib/python3.6', '/Users/chrisg/miniconda3/envs/drlnd/lib/python3.6/lib-dynload', '', '/Users/chrisg/miniconda3/envs/drlnd/lib/python3.6/site-packages', '/Users/chrisg/miniconda3/envs/drlnd/lib/python3.6/site-packages/IPython/extensions', '/Users/chrisg/.ipython', '/Users/chrisg/dev/deep-reinforcement-continuous-control']


In [3]:
from unityagents import UnityEnvironment
import numpy as np

In [4]:
env = UnityEnvironment(file_name='Reacher20.app', no_graphics=True)

#udacity env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [5]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
brain_name

'ReacherBrain'

In [6]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


In [7]:
from ddpg_agent import Agent

agent = Agent(state_size, action_size, random_seed=2)

In [8]:
from collections import deque
import matplotlib.pyplot as plt
import torch
from timeit import default_timer as timer

# Udacity GPU use max_t=1000
# target_window, keep at 100 to solve environment
# target_score for SMA, keep at 30 to solve environment
def ddpg(n_episodes=500, max_t=1000, print_every=10, target_window=100, target_score=30):
    scores_deque = deque(maxlen=target_window)
    scores_total = []
    train_every = 20
    train_iterations = 10
    
    for i_episode in range(1, n_episodes+1):
        
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        scores = np.zeros(num_agents)
        agent.reset()
         
        start = timer()
        for t in range(max_t):
            
            actions = agent.act(states)
            #print(actions)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations #[0]
            rewards = env_info.rewards #[0]
            dones = env_info.local_done #[0]
            
            train = not bool(t % train_every)
            
            for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):         
                agent.step(state, action, reward, next_state, done, train, train_iterations)
            states = next_states
            scores += rewards
            #print(scores)
            if np.any(dones):
                break 
        end = timer()
        delta = end - start
        scores_mean = np.mean(scores)
        scores_deque.append(scores_mean)
        scores_total.append(scores_mean)
        sma = np.mean(scores_deque)
        print('\rEpisode {}\tAverage Score: {:.2f}\tLast Mean: {:.2f}'.format(i_episode, sma, scores_mean), end="")
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if i_episode % print_every == 0:
            sma_slice = [ int(s*100)/100.0 for s in scores_deque][-5:]
            print('\rEpisode {}\tAverage Score: {:.2f}\t({:.0f} sec)\t{}'.format(i_episode, sma, delta, sma_slice))
              
        sma_bool = [s >= target_score for s in scores_deque]
        #print(scores_deque)
        if np.all(sma_bool) and len(sma_bool) >= target_window: 
            print("\n*** Environment Solved ***")
            break
                
            
    return scores_total, scores_deque

scores, sma = ddpg()
print([ int(s*100)/100.0 for s in sma])
#print(scores)
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

/Users/chrisg/miniconda3/envs/drlnd/lib/python3.6/site-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Episode 1	Average Score: 0.08	(73 sec)	[0.08]
Episode 2	Average Score: 0.14	(79 sec)	[0.08, 0.19]
Episode 3	Average Score: 0.17	(78 sec)	[0.08, 0.19, 0.23]
Episode 4	Average Score: 0.24	(79 sec)	[0.08, 0.19, 0.23, 0.43]
Episode 5	Average Score: 0.25	(79 sec)	[0.08, 0.19, 0.23, 0.43, 0.3]
Episode 6	Average Score: 0.32	(79 sec)	[0.08, 0.19, 0.23, 0.43, 0.3, 0.64]


KeyboardInterrupt: 

When finished, you can close the environment.

In [ ]:
env.close()